In [16]:

import geemap
import ee

# Initialize the Earth Engine module
ee.Initialize()

# Define a region of interest (Sicily)
sicily = ee.Geometry.Rectangle([11.0, 36.0, 15.5, 38.5])  # [minlon, minlat, maxlon, maxlat]

# Load Sentinel-2 data and filter by date and region
sentinel2 = ee.ImageCollection('COPERNICUS/S2') \
            .filterBounds(sicily) \
            .filterDate('2023-01-01', '2023-12-31') \
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))

# Function to create a false-color composite (B8, B4, B3)
def falseColorComposite(image):
    return image.select(['B8', 'B4', 'B3']).clip(sicily)

# Map the false-color function over the image collection
false_color_collection = sentinel2.map(falseColorComposite)

# Select the first image in the collection
first_image = false_color_collection.first()

# Visualization parameters for the false-color composite
vis_params_false_color = {
    'min': 0,
    'max': 3000,
    'bands': ['B8', 'B4', 'B3'],
    'gamma': 1.4
}

# Load a DEM dataset for Sicily
dem = ee.Image('USGS/SRTMGL1_003').clip(sicily)

# Visualization parameters for the DEM
vis_params_dem = {
    'min': 0,
    'max': 3000,
    'palette': ['000000', 'ffffff']  # Black to white color palette for DEM
}

# Create a map centered on Sicily
Map = geemap.Map(center=[37.6, 14.0], zoom=8)

# Add the false-color composite to the map
Map.addLayer(first_image, vis_params_false_color, 'False-Color Composite')

# Add the DEM to the map
Map.addLayer(dem, vis_params_dem, 'DEM')
Map.add_colorbar(vis_params_dem, label="Elevation (m)", layer_name="SRTM DEM")
Map



Map(center=[37.6, 14.0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(…

In [17]:
import geemap
import ee

# Initialize the Earth Engine module
ee.Initialize()

# Define a region of interest (Sicily)
sicily = ee.Geometry.Rectangle([11.0, 36.0, 15.5, 38.5])  # [minlon, minlat, maxlon, maxlat]

# Load Sentinel-2 data and filter by date and region
sentinel2 = ee.ImageCollection('COPERNICUS/S2') \
            .filterBounds(sicily) \
            .filterDate('2023-01-01', '2023-12-31') \
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))

# Function to create a false-color composite (B8, B4, B3)
def falseColorComposite(image):
    return image.select(['B8', 'B4', 'B3']).clip(sicily)

# Map the false-color function over the image collection
false_color_collection = sentinel2.map(falseColorComposite)

# Select the first image in the collection
first_image = false_color_collection.first()

# Visualization parameters for the false-color composite
vis_params_false_color = {
    'min': 0,
    'max': 3000,
    'bands': ['B8', 'B4', 'B3'],
    'gamma': 1.4
}

# Load a DEM dataset for Sicily
dem = ee.Image('USGS/SRTMGL1_003').clip(sicily)

# Visualization parameters for the DEM (change color palette)
vis_params_dem = {
    'min': 0,
    'max': 3000,
    'palette': ['blue', 'cyan', 'green', 'yellow', 'orange', 'red']  # Custom color palette for DEM
}

# Create a map centered on Sicily
Map = geemap.Map(center=[37.6, 14.0], zoom=8)

# Add the false-color composite to the map
Map.addLayer(first_image, vis_params_false_color, 'False-Color Composite')

# Add the DEM to the map with the custom color palette
Map.addLayer(dem, vis_params_dem, 'DEM')
Map.add_colorbar( vis_params_dem, label="Elevation (m)", layer_name="SRTM DEM")

# Display the map
Map


Map(center=[37.6, 14.0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(…

No such comm: 7f334b6c65c14728ab5adc6f45421a1a


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [19]:
Map.user_roi_coords()

[13.7865, 37.5849, 14.5033, 38.0351]

In [27]:
roi= ee.Geometry.Rectangle([13.7865, 37.5849, 14.5033, 38.0351])


In [35]:
roi = ee.Geometry.Rectangle(13.7865, 37.5849, 14.5033, 38.0351)

# Function to create and display a map with a transparent fill for the ROI
def map_with_transparent_fill(roi):
    # Create a map centered on Sicily
    Map = geemap.Map(center=[38.0, 13.75], zoom=8)

    # Add the ROI to the map with transparent fill
    Map.addLayer(roi, {'color': 'FF0000', 'fillColor': '', 'width': 0}, 'Sicily Region')

    # Display the map
    Map.addLayerControl()  # Add layer control widget for user interaction
    return Map

# Display the map with transparent fill for the ROI
map_with_transparent_fill(roi)


Map(center=[38.0, 13.75], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI…

In [40]:
import ee
import geemap

# Initialize Earth Engine
ee.Initialize()

# Load the dataset
dataset = ee.ImageCollection('ESA/WorldCereal/2021/MODELS/v100')

# Set satellite background
Map = geemap.Map()
Map.setOptions('SATELLITE')

# Function to mask the "other" class (value 0)
def mask_other(img):
    return img.updateMask(img.neq(0))

# Apply the mask_other function to the collection
dataset = dataset.map(mask_other)

# --------------------------------------------------
# Basic example for a global mosaic of temporary crops
# --------------------------------------------------

# Get a global mosaic for all agro-ecological zone (AEZ) of temporary crops
temporarycrops = dataset.filter(ee.Filter.eq('product', 'temporarycrops')).mosaic()

# Visualization specifics
visualization_class = {
    'bands': ["classification"],
    'max': 100,
    'palette': ["#ff0000"]
}

visualization_conf = {
    'bands': ['confidence'],
    'min': 0,
    'max': 100,
    'palette': ['be0000', 'fff816', '069711'],
}

# Show global classification mosaic
Map.centerObject(temporarycrops)
Map.addLayer(temporarycrops, visualization_class, 'Temporary crops')

# By default don't show confidence layer
Map.addLayer(temporarycrops, visualization_conf, 'Temporary crops confidence', False)

# --------------------------------------------------
# Advanced example for tc-maize-main season products
# in a specific AEZ
# --------------------------------------------------

# Filter on AEZ and season
tc_maize_main_46172 = dataset.filter(ee.Filter.eq('season', 'tc-maize-main')) \
                             .filter(ee.Filter.eq('aez_id', 46172))

# Get the different products
maize = tc_maize_main_46172.filter(ee.Filter.eq('product', 'maize'))
irrigation = tc_maize_main_46172.filter(ee.Filter.eq('product', 'irrigation'))

# Visualization specifics
visualization_maize = {
    'bands': ["classification"],
    'max': 100,
    'palette': ["#ebc334"]
}

visualization_irrigation = {
    'bands': ["classification"],
    'max': 100,
    'palette': ["#2d79eb"]
}

# Show maize and irrigation classification
Map.addLayer(maize, visualization_maize, 'Maize')
Map.addLayer(irrigation, visualization_irrigation, 'Active irrigation')

# Add legends
legend_keys = ['Temporary crops', 'Maize', 'Active irrigation']
legend_colors = ['#ff0000', '#ebc334', '#2d79eb']
Map.add_legend(legend_keys=legend_keys, legend_colors=legend_colors)

# Uncomment the line below to zoom to a region
# where maize, other crops and active irrigation are visible
Map.setCenter(-0.9911, 43.5017, 12)

# Display the map
Map.addLayerControl()  # Add layer control widget for user interaction
Map


Map(center=[43.5017, -0.9911], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDa…

In [42]:
import ee
import geemap

# Initialize Earth Engine
ee.Initialize()

# Load the dataset
dataset = ee.ImageCollection('ESA/WorldCereal/2021/MODELS/v100')

# Define a geometry for Sicily
sicily_geometry = ee.Geometry.Polygon(
    [[[11.472416, 36.619987],
      [11.472416, 38.288231],
      [15.689207, 38.288231],
      [15.689207, 36.619987]]])

# Filter dataset to Sicily
dataset_sicily = dataset.filterBounds(sicily_geometry)

# Set satellite background
Map = geemap.Map()
Map.setOptions('SATELLITE')

# Function to mask the "other" class (value 0)
def mask_other(img):
    return img.updateMask(img.neq(0))

# Apply the mask_other function to the collection
dataset_sicily = dataset_sicily.map(mask_other)

# --------------------------------------------------
# Basic example for a global mosaic of temporary crops
# --------------------------------------------------

# Get a global mosaic for all agro-ecological zone (AEZ) of temporary crops in Sicily
temporarycrops_sicily = dataset_sicily.filter(ee.Filter.eq('product', 'temporarycrops')).mosaic()

# Visualization specifics
visualization_class = {
    'bands': ["classification"],
    'max': 100,
    'palette': ["#ff0000"]
}

visualization_conf = {
    'bands': ['confidence'],
    'min': 0,
    'max': 100,
    'palette': ['be0000', 'fff816', '069711'],
}

# Add legends
legend_keys = ['Temporary crops', 'Maize', 'Active irrigation']
legend_colors = ['#ff0000', '#ebc334', '#2d79eb']
Map.add_legend(legend_keys=legend_keys, legend_colors=legend_colors)

# Show global classification mosaic for Sicily
Map.centerObject(sicily_geometry)
Map.addLayer(temporarycrops_sicily, visualization_class, 'Temporary crops - Sicily')

# By default don't show confidence layer
Map.addLayer(temporarycrops_sicily, visualization_conf, 'Temporary crops confidence - Sicily', False)

# Display the map
Map.addLayerControl()  # Add layer control widget for user interaction
Map


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…